<a href="https://colab.research.google.com/github/tincorpai/Recommendation-System/blob/master/KNN_based_Coll_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/Shared drives/machinelearning/deep_learning_with_python/recommendation_python')

In [ ]:
data = pd.read_csv("data_clean.csv")
data.head(2)

,Unnamed: 0,userId,movieId,rating,title,genres,TotalRatingCount
0,0,1,2,3.5,Jumanji (1995),Adventure|Children|Fantasy,112
1,1,5,2,3.0,Jumanji (1995),Adventure|Children|Fantasy,112


In [ ]:
len(set(data['title'])), data.shape

(8225, (100000, 7))

In [ ]:
data.columns 


Index(['Unnamed: 0', 'userId', 'movieId', 'rating', 'title', 'genres',
       'TotalRatingCount'],
      dtype='object')

In [ ]:
data = data.drop(columns = 'Unnamed: 0')


In [ ]:
data.columns

Index(['userId', 'movieId', 'rating', 'title', 'genres', 'TotalRatingCount'], dtype='object')

In [ ]:
pivot_table = data.pivot_table(index = 'title', columns='userId', values='rating').fillna(0)

In [ ]:
pivot_table.shape

(8225, 702)

In [ ]:
pivot_table.values

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
pivot_table.values

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
features_matrix = csr_matrix(pivot_table.values)

In [ ]:
features_matrix

<8225x702 sparse matrix of type '<class 'numpy.float64'>'
	with 99997 stored elements in Compressed Sparse Row format>

We build KNN collaborative model 

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
model = NearestNeighbors(metric='cosine', algorithm='brute')

In [ ]:
model.fit(features_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
pivot_table.shape[0]

8225

In [ ]:
#We try to recommend based on the movie index
query_index = np.random.choice(pivot_table.shape[0])

In [ ]:
query_index

4705

In [ ]:
query_ind = np.random.choice(pivot_table.shape[1])

In [ ]:
#This says there are 620 unique users that have voted for the movie
query_ind

245

In [ ]:
pivot_table.iloc[4209:].values.reshape(1, -1)

array([[0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
distances, indices = model.kneighbors(pivot_table.iloc[query_index, :].values.reshape(1, -1), n_neighbors=8)

In [ ]:
distances

array([[0.        , 0.18324605, 0.26278792, 0.28014396, 0.3598156 ,
        0.3598156 , 0.3598156 , 0.3598156 ]])

In [ ]:
indices.flatten()

array([3002, 2795, 3604, 6184, 2815, 4470, 2249, 7052])

These shows all the movie IDs having all these distances

In [ ]:
pivot_table.index[3002]

'Going My Way (1944)'

In [ ]:
pivot_table.iloc[indices.flatten()].index

Index(['Going My Way (1944)', 'Friendly Persuasion (1956)',
       'I'll Be Home For Christmas (1998)', 'Road to Singapore (1940)',
       'From the Terrace (1960)', 'Lost in Yonkers (1993)',
       'Easter Parade (1948)', 'Sunshine Boys, The (1975)'],
      dtype='object', name='title')

Using kernel based recommendation system, he might get all these movies for the recommendation.

In [ ]:
distances.flatten()

array([0.        , 0.18324605, 0.26278792, 0.28014396, 0.3598156 ,
       0.3598156 , 0.3598156 , 0.3598156 ])

In [ ]:
for i in range(0, len(distances.flatten())):
  if i == 0:
    print('Recommendations for {0}'.format(pivot_table.index[query_index]))
  else:
    print('{} : {}, with a distance of {}'.format(i, pivot_table.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Going My Way (1944)
1 : Friendly Persuasion (1956), with a distance of 0.18324605213729983
2 : I'll Be Home For Christmas (1998), with a distance of 0.26278791724696693
3 : Road to Singapore (1940), with a distance of 0.28014395681439486
4 : From the Terrace (1960), with a distance of 0.35981560033552007
5 : Lost in Yonkers (1993), with a distance of 0.35981560033552007
6 : Easter Parade (1948), with a distance of 0.35981560033552007
7 : Sunshine Boys, The (1975), with a distance of 0.35981560033552007
